<h2 style = "text-align: center;">Advanced Data Science for Traffic and Transportation Engineering</h2>

<h3 style = "text-align: center;">Determine the Position of Road Inspectors - Rijkswaterstaat</h3><br><br>

<div style="display:flex">
     <div style="flex:1;padding-right:10px;">
          <img src="./Images/rws.jpg" width="260"/>
     </div>
     <div style="flex:1;padding-left:10px;">
          <img src="./Images/tud.png" width="200"/>
     </div>
</div>
<h3 style = "text-align: left;">Group 3</h3>

<ul>
     <li>Yiman Bao (5691648)</li>
     <li>Juan Camargo Fonseca (5834112))</li>
     <li>Tijmen Hoedjes (4959183)</li>
     <li>Max Lange (5169402)</li>
     <li>Wail Abdellaoui (5130654)</li>
</ul>

### **Content** ###

**This notebook is structured as follows:**

<b>1. Introduction</b>
<b><p style="margin-left: 40px">1.1 Research Objective and Research Questions</p></b>
<b><p style="margin-left: 40px">1.2 Tech Stack</p></b>
<b><p style="margin-left: 40px">1.3 Project Management</p></b>

<b>2. Data Story</b>
<b><p style="margin-left: 40px">2.1 Data Overview</p></b>
<b><p style="margin-left: 40px">2.2 Data Filtering and Preprocessing</p></b>
<b><p style="margin-left: 40px">2.3 Streamlit App</p></b>

<b>3. Algorithm</b>
<b><p style="margin-left: 40px">3.1 Algorithm Requirements</p></b>
<b><p style="margin-left: 40px">3.2 Methodology - Overview</p></b>

<b>4. Results</b>

<b>5. Validation</b>

<b>6. Conclusion</b>





### 1. Introduction ###

Have you ever been involved in a road incident in the Netherlands? Did you ever were late to an appointment or event because an incident occurred? If you answered yes, it is most likely that you needed a road inspector to make your life easier. As you may have noticed by now, road inspectors are important to ensure safety on highways and a smooth traffic flow. When incident occur, they appear and make sure that the traffic can quickly resume. Therefore, you can imagine it is important for them to show up as soon as possible when incidents happen. In consequence, an optimal distribution of the inspectors to make sure that they will arrive shortly after the incident occur is necessary, in this document we discuss the considerations made for reaching this objective, and the implications of this for road users in the Netherlands.

#### 1.1 Research Objective and Research Questions ####

The research objective is to find out the optimal location of inspectors in the Dutch road network, such that travel time to the incidents is minimized. For reaching this objective the following research question(s) are proposed. <br><br><b>Main Research Question:</b> What would be the optimal location of road inspectors in the Netherlands, such that travel times to incidents are minimized?

<b>Sub questions:</b>
<ol>
    <li> To what extent the data provided by Rijkswaterstaat is useful for the research objective? </li>
    <li> How do the incidents distribute in time and space in the Dutch network? </li>
    <li> What type of method is suitable for evaluating the accident probabilities? </li>
    <li> What are the locations in which accidents are more likely to happen? </li>
    <li> What variables would affect the response time of the inspectors? </li>
    <li> How to calculate the response time of the inspectors?  </li>
    <li> What is the optimal number of road inspectors needed?  </li>
    <li> What is the optimal location for the road inspectors?  </li>
</ol>
    
#### 1.2 Tech Stack ####

Given the requirement of this project, the following is the tech stack that will be used for reaching the research objective.

<b> Coding: </b>The main programming language used will be Python, and necessary packages for data analytics such as Pandas, NumPy, GeoPandas, SciPy and scikit-learn will be imported as necessary.

<b> Visualization: </b>For standard data visualization, Python libraries such as Matplotlib, Plotly and Seaborn will be employed. For Geospatial visualization, additional tools such as Folium and Rasterio are considered.

<b> Version Control: </b>Git and GitHub will be used for code tracking and collaboration.

<b> Communication: </b>Weekly in-person meetings are planned. However, Microsoft Teams can be used as necessary.

<b> Documentation: </b>Jupyter Notebooks will be used for documenting code, analysis, results and conclusions.

The following code block imports the necessary packages and libraries.

In [2]:
!streamlit run Homepage.py

^C
